--- Day 16: The Floor Will Be Lava ---
With the beam of light completely focused somewhere, the reindeer leads you deeper still into the Lava Production Facility. At some point, you realize that the steel facility walls have been replaced with cave, and the doorways are just cave, and the floor is cave, and you're pretty sure this is actually just a giant cave.

Finally, as you approach what must be the heart of the mountain, you see a bright light in a cavern up ahead. There, you discover that the beam of light you so carefully focused is emerging from the cavern wall closest to the facility and pouring all of its energy into a contraption on the opposite side.

Upon closer inspection, the contraption appears to be a flat, two-dimensional square grid containing empty space (.), mirrors (/ and \), and splitters (| and -).

The contraption is aligned so that most of the beam bounces around the grid, but each tile on the grid converts some of the beam's light into heat to melt the rock in the cavern.

You note the layout of the contraption (your puzzle input). For example:

.|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|....
The beam enters in the top-left corner from the left and heading to the right. Then, its behavior depends on what it encounters as it moves:

If the beam encounters empty space (.), it continues in the same direction.
If the beam encounters a mirror (/ or \), the beam is reflected 90 degrees depending on the angle of the mirror. For instance, a rightward-moving beam that encounters a / mirror would continue upward in the mirror's column, while a rightward-moving beam that encounters a \ mirror would continue downward from the mirror's column.
If the beam encounters the pointy end of a splitter (| or -), the beam passes through the splitter as if the splitter were empty space. For instance, a rightward-moving beam that encounters a - splitter would continue in the same direction.
If the beam encounters the flat side of a splitter (| or -), the beam is split into two beams going in each of the two directions the splitter's pointy ends are pointing. For instance, a rightward-moving beam that encounters a | splitter would split into two beams: one that continues upward from the splitter's column and one that continues downward from the splitter's column.
Beams do not interact with other beams; a tile can have many beams passing through it at the same time. A tile is energized if that tile has at least one beam pass through it, reflect in it, or split in it.

In the above example, here is how the beam of light bounces around the contraption:

>|<<<\....
|v-.\^....
.v...|->>>
.v...v^.|.
.v...v^...
.v...v^..\
.v../2\\..
<->-/vv|..
.|<<<2-|.\
.v//.|.v..
Beams are only shown on empty tiles; arrows indicate the direction of the beams. If a tile contains beams moving in multiple directions, the number of distinct directions is shown instead. Here is the same diagram but instead only showing whether a tile is energized (#) or not (.):

######....
.#...#....
.#...#####
.#...##...
.#...##...
.#...##...
.#..####..
########..
.#######..
.#...#.#..
Ultimately, in this example, 46 tiles become energized.

The light isn't energizing enough tiles to produce lava; to debug the contraption, you need to start by analyzing the current situation. With the beam starting in the top-left heading right, how many tiles end up being energized?

In [1]:
import get_data

import numpy as np

In [2]:
# input data as np matrix 
data_matrix = get_data.file_as_matrix(day_number = 16)

# reflections dictionary converting incomming directions and mirrors to the corresponding output direction
REFLECTIONS_DICT = {
    # from left
    ((0,1), "/") : (-1,0),
    ((0,1), "\\") : (1,0),
    
    # from right
    ((0,-1), "/") : (1,0),
    ((0,-1), "\\") : (-1,0),
    
    # from top
    ((1,0), "/") : (0,-1),
    ((1,0), "\\") : (0,1),
    
    # from bottom
    ((-1,0), "/") : (0,1),
    ((-1,0), "\\") : (0,-1),
}

In [11]:
# helper functions

def point_in_matrix(point, matrix):
    
    if 0 <= point[0] < matrix.shape[0] and 0 <= point[1] < matrix.shape[1]:
        return True
    
    return False

def get_direction(direction, mirror):
    
    return np.array(REFLECTIONS_DICT[(tuple(direction), mirror)])

def mirror_in_line(direction, position):
    
    if (position == "-" and direction[0] == 0
        or position == "|" and direction[1] == 0): 
        
        return True
    
    return False

def split_laser(direction):
    if direction[0] == 0:
        return [np.array([1,0]), np.array([-1,0])]
    elif direction[1] == 0:
        return [np.array([0,1]), np.array([0,-1])]
    else:
        print("Something went wrong. The direction uses multiple axis")

def follow_light(position, direction, matrix, solution_matrix, history):
    
    while point_in_matrix(position, matrix):
        
        pos = tuple(position)
        
        if matrix[pos] != '.':

            if matrix[pos] in ['/', '\\']:

                direction = get_direction(direction, matrix[pos])

            elif matrix[pos] in ['-', '|']:

                if not mirror_in_line(direction, matrix[pos]):
                    
                    # split directions
                    new_directions_list = split_laser(direction)
                    
                    # call follow_light function with one direction
                    # make a step in this direction (to not repeat the same loop over and over when calling the funciton)
                    follow_light(position + new_directions_list[1], new_directions_list[1], data_matrix, solution_matrix, history)
                    
                    # continue this loop with other direction
                    direction = new_directions_list[0]
                    
            if (tuple(direction), pos) in history: break
            else: history.append((tuple(direction), pos))
            
        solution_matrix[pos] = True
        position += direction
        
    return solution_matrix

In [12]:
def solve_puzzle(starting_point, starting_direction):

    return np.sum(follow_light(starting_point, starting_direction, data_matrix, np.full(data_matrix.shape, False), []))

solve_puzzle(np.array([0,0]), np.array([0,1]))

7996

7996 is correct

In [19]:
def solve_second_part():
    
    best_score = 0
    
    for row_index in range(data_matrix.shape[0]):
        
        left = solve_puzzle(np.array([row_index,0]), np.array([0,1]))
        right = solve_puzzle(np.array([row_index, data_matrix.shape[0] - 1]), np.array([0,-1]))
        
        if left > best_score:
            best_score = left
        if right > best_score:
            best_score = right
            
    for col_index in range(data_matrix.shape[1]):
        
        top = solve_puzzle(np.array([0, col_index]), np.array([1,0]))
        bottom = solve_puzzle(np.array([data_matrix.shape[1] - 1, col_index]), np.array([-1,0]))
        
        if top > best_score:
            best_score = top
        if bottom > best_score:
            best_score = bottom
            
    return best_score

solve_second_part()

8239

8239 is correct